In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
c = 0;
for dirname, _, filenames in os.walk('/kaggle/input/noised-dataset'):
    for filename in filenames:
        path = (os.path.join(dirname, filename))
        c+=1
c


**Have to use Wave-UNET**

In [ ]:
clean = '/kaggle/input/cleaned-dataset'
noise = '/kaggle/input/noised-dataset'

In [ ]:
import time;
st = time.time();
file_path = '/kaggle/input/clean-audio/Clean/03-01-01-01-01-01-03.wav'
audio,sr = librosa.load(file_path)
print(time.time()-st)
duration = librosa.get_duration(y=audio, sr=sr)
print(duration,sr)
22050*3.55

In [ ]:
clean_list = []
noise_list = []
duration = 2.972154
sr = 22050
for dirname, _, filenames in os.walk('/kaggle/input/clean-audio'):
    for filename in filenames:
        path_clean = os.path.join(dirname, filename)
        audio,_ = librosa.load(path_clean, sr=sr, duration=duration)
        clean_list.append(audio)
        
        filename = filename[:len(filename)-4]
        filename += '_noisy.wav'
        path_noisy = os.path.join(noise,filename)
        audio,_ = librosa.load(path_noisy, sr=sr, duration=duration)
        noise_list.append(audio)

In [ ]:
print(len(noise_list[0]),len(clean_list))

In [ ]:
noise_list = np.array(noise_list)
clean_list = np.array(clean_list)

# Training the U-NET MODEL

In [ ]:
import tensorflow as tf

def conv_block(inputs, num_filters):
    x = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.01)(x)

    x = tf.keras.layers.Conv2D(num_filters, 3, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.01)(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = tf.keras.layers.MaxPool2D((2,2))(x)
    return x, p

def decoder_block(inputs, skip, num_filters):
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x = tf.keras.layers.Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = tf.keras.layers.Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)   # 256 x 256
    s2, p2 = encoder_block(p1, 128)     # 128 x 128
    s3, p3 = encoder_block(p2, 256)     # 64 x 64
     # 32 x 32

    # Bridge
    b1 = conv_block(p3, 1024)           # 16 x 16

    # Decoder
#     d1 = decoder_block(b1, s4, 512)     # 32 x 32
    d2 = decoder_block(b1, s3, 256)     # 64 x 64
    d3 = decoder_block(d2, s2, 128)     # 128 x 128
    d4 = decoder_block(d3, s1, 64)      # 256 x 256

    outputs = tf.keras.layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
    model = tf.keras.models.Model(inputs, outputs, name="UNET")
    return model

input_shape = (64, 128, 1)
model = build_unet(input_shape)
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

Xnoisy_train, Xnoisy_val, Yclean_train, Yclean_val = train_test_split(noise_list, clean_list, test_size=0.2, random_state=0)

In [ ]:
Xnoisy_train = Xnoisy_train[..., np.newaxis]
Yclean_train = Yclean_train[..., np.newaxis]
Xnoisy_val = Xnoisy_val[..., np.newaxis]
Yclean_val = Yclean_val[..., np.newaxis]

In [ ]:
batch_size = 8
epochs = 20

history = model.fit(Xnoisy_train, Yclean_train, validation_data=(Xnoisy_val, Yclean_val), epochs=20)

In [ ]:
# print(history.history.keys())  # Print available keys

# # Plot training history
# import matplotlib.pyplot as plt

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

In [ ]:
# model.summary()
sr = 22050

**CLEAN AUDIO FILE EXTRACTION**
* Predicting the clean audio
* Saving the audio for futher transciption and summarization

In [ ]:
from IPython.display import Audio 
test_path = '/kaggle/input/noisy-audio/Noisy/03-01-01-01-01-01-03_noisy.wav'
# audio,_ = librosa.load(test_path)
# pred = model.predict(audio)

Xaudio, sr = librosa.load(test_path, sr=sr,duration=duration)  # Load with original sample rate
Xaudio = np.array(Xaudio)
print(Xaudio.shape)
Xaudio = Xaudio[..., np.newaxis]
print(Xaudio)
Xaudio = Xaudio.reshape(-1, 65536, 1)

print(Xaudio.shape)
# audio_input = audio.reshape(1, -1)  
# (None, 65536, 1)
prediction = model.predict(Xaudio)
print(prediction)

output_path = '/kaggle/working/clean_audio.wav'
sf.write(output_path, prediction.squeeze(), sr)

Audio(Xaudio.squeeze(), rate=sr)

# WHISPER MODEL -- Transcription

In [ ]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3")

In [ ]:
# input_features = processor(
#     waveform, sampling_rate=sampling_rate, return_tensors="pt"
# ).input_features

# # Generate token ids
# predicted_ids = model.generate(input_features)

# # Decode token ids to text
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

# transcription[0]

In [ ]:
!pip install git+https://github.com/openai/whisper

In [ ]:
import whisper

model = whisper.load_model("base")

audio = whisper.load_audio(output_path)
audio = whisper.pad_or_trim(audio)

mel = whisper.log_mel_spectrogram(audio).to(model.device)

_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

print(result.text)

**result.text is my transcribed text**

In [ ]:
text_transcribed = result.text

# Summarisation model FLAN-T5

In [ ]:
!pip install --upgrade transformers
import transformers

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-base")

**CHECKING MY TRANSCRIPTION MODEL AND TOKENIZER**

In [ ]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

***ONE SHOT INFERENCE OR MULTI INFERNECE WAS NOT HELPING***

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

example_text = "The quick brown fox jumps over the lazy dog. This sentence is often used to test typewriters and keyboards because it contains every letter of the alphabet."
example_summary = "A sentence used for testing typewriters and keyboards."

prompt = f"""
Summarize the following text.

Example:
Text: {example_text}
Summary: {example_summary}

Text: {text_transcribed}
Summary:
    """

print(prompt)

inputs = tokenizer(prompt, return_tensors='pt')

output = tokenizer.decode(
  model.generate(
    inputs["input_ids"],
    max_new_tokens=50,
  )[0],
  skip_special_tokens=True
)

print("Predicted summary = ", output)


# ____END OF PROJECT_____